In [ ]:
!pip install "gensim>=4.0.0"

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.w2v || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.w2v.gz && gunzip heise-articles-2020.w2v.gz")
    os.system("test -f heise-articles-2020.ft || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.ft.gz && gunzip heise-articles-2020.ft.gz")
    os.system("test -f heise-articles-2020.glove.txt || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.glove.txt.gz && gunzip heise-articles-2020.glove.txt.gz")
    w2v_file = "heise-articles-2020.w2v"
    ft_file = "heise-articles-2020.ft"
    glove_file = "heise-articles-2020.glove.txt"
else:
    w2v_file = "../99_Common/heise-articles-2020.w2v"
    ft_file = "../99_Common/heise-articles-2020.ft"
    glove_file = "../99_Common/heise-articles-2020.glove.txt"

In [ ]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format(w2v_file)

In [ ]:
ft = KeyedVectors.load_word2vec_format(ft_file)

In [ ]:
glove = KeyedVectors.load_word2vec_format(glove_file, no_header=True)

In [ ]:
len(w2v.key_to_index)

In [ ]:
if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT keywords FROM articles WHERE datePublished<'2021-01-01'", sql)

In [ ]:
from collections import Counter
keywords = Counter([keyword for keywords in df["keywords"] for keyword in str(keywords).split(", ")])

In [ ]:
top_keywords = [k[0] for k in keywords.most_common(100) if not " " in k[0]][0:50]

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def plot_wv(model, words):
    wr = []
    wk = []
    for w in words:
        # manche Wörter könnten fehlen
        try:
            wr.append(model[w.lower()])
            wk.append(w)
        except:
            pass

    tsne = TSNE(n_components=2, random_state=42).fit_transform(wr)
    tsne_df = pd.DataFrame(tsne, columns = ["x", "y"])
    
    ax = tsne_df.plot.scatter(x='x', y='y', figsize=(16, 9))

    for i, txt in enumerate(wk):
        ax.annotate(txt, (tsne_df.x[i], tsne_df.y[i]))

In [ ]:
plot_wv(w2v, top_keywords)

In [ ]:
plot_wv(ft, top_keywords)

In [ ]:
plot_wv(glove, top_keywords)

In [ ]:
!pip install networkx

In [ ]:
import networkx as nx

def graph(model, w0):
    G=nx.Graph()
    G.add_node(w0)
    for (w1, s1) in model.most_similar(w0):
        G.add_node(w1)
        G.add_edge(w0, w1, weight=s1)
        for (w2, s2) in model.most_similar(w1):
            G.add_node(w2)
            G.add_edge(w1, w2, weight=s2)
            
    return G

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(3,figsize=(12,12)) 
G = graph(w2v, "apple")
nx.draw_kamada_kawai(G, with_labels=True, font_weight='bold')

In [ ]:
plt.figure(3,figsize=(12,12)) 
G = graph(ft, "apple")
nx.draw_kamada_kawai(G, with_labels=True, font_weight='bold')

In [ ]:
plt.figure(3,figsize=(12,12)) 
G = graph(glove, "apple")
nx.draw_kamada_kawai(G, with_labels=True, font_weight='bold')